In [ ]:
pip install -q tf-models-official==2.4.0

     |████████████████████████████████| 1.1 MB 10.5 MB/s 
     |████████████████████████████████| 211 kB 24.0 MB/s 
     |████████████████████████████████| 37.1 MB 50 kB/s 
     |████████████████████████████████| 99 kB 6.6 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 1.1 MB 40.6 MB/s 
     |████████████████████████████████| 352 kB 49.3 MB/s 
     |████████████████████████████████| 636 kB 39.6 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/데이터크리에이터캠프/nlp/task1/data'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from collections import Counter
import pandas as pd
import re
import os

In [ ]:
# 데이터 불러오기
train_path = os.path.join(path, 'train.csv')
test_path = os.path.join(path, 'test_x.csv')
sub_path = os.path.join(path, 'sample_submission.csv')

train = pd.read_csv(train_path, index_col=0)
test = pd.read_csv(test_path, index_col=0)
sub = pd.read_csv(sub_path, index_col=0)

https://www.tensorflow.org/official_models/fine_tuning_bert 참고

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [ ]:
# bert의 config, vocab, pre-trained weight를 담고 있는 경로
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [ ]:
# bert 토크나이저
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
     do_lower_case=True)

print("Vocab size:", len(tokenizer.vocab))

Vocab size: 30522


In [ ]:
# 전처리 함수
def preprocessing(text):
  text = re.sub("<br\\s*/?>", ' ', text)
  text = re.sub("[^a-zA-Z\']", ' ', text)
  return text

In [ ]:
# 전처리. 영어 이외의 문자 모두 제거
x_train = train['text'].copy()
y_train = train['author'].copy()
y_train = tf.keras.utils.to_categorical(y_train)

for i in x_train.index:
    text = preprocessing(x_train[i])
    x_train[i] = text

In [ ]:
# bert train을 위한 encoding
def encode_sentence(s, tokenizer):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]') # 모든 문장은 SEP로 끝남
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(x_train, tokenizer):
  num_examples = len(x_train)

  tf_text = tf.ragged.constant([
      encode_sentence(s, tokenizer)
      for s in np.array(x_train.tolist())])

  # 모든 문장은 CLS로 시작
  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*tf_text.shape[0]
  input_word_ids = tf.concat([cls, tf_text], axis=-1)

  # 내용(pad x) 인 부분을 알려줌(1, 0)
  input_mask = tf.ones_like(input_word_ids).to_tensor()

  # 실제 내용의 위치
  type_cls = tf.zeros_like(cls)
  type_text = tf.zeros_like(tf_text)
  input_type_ids = tf.concat(
      [type_cls, type_text], axis=-1).to_tensor()

  # bert input
  inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [ ]:
train_set = bert_encode(x_train, tokenizer)

In [ ]:
for key, value in train_set.items():
  print(f'{key:15s} shape: {value.shape}')

input_word_ids  shape: (54879, 501)
input_mask      shape: (54879, 501)
input_type_ids  shape: (54879, 501)


In [ ]:
import json

# bert 설정
bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

config_dict

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [ ]:
# bert 분류기, encoder load
bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=5)

In [ ]:
# bert의 encoder는 생성될 때 weight가 랜덤 초기화
# pre train된 weight를 다시 저장
checkpoint = tf.train.Checkpoint(encoder=bert_encoder)
checkpoint.read(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()

In [ ]:
# 훈련 설정
# Set up epochs and steps
epochs = 10
batch_size = 32

train_data_size = len(y_train)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

In [ ]:
bert_classifier.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
# train 시간이 1 에포크 당 3 시간
bert_classifier.fit(
      train_set, y_train,
      batch_size=32,
      epochs=epochs)

Epoch 1/10


ResourceExhaustedError: ignored